In [1]:
import pandas as pd
import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
global_music_file_paths = glob2.glob('spotify za v2/*.csv')

In [3]:
print(len(global_music_file_paths))

345


In [4]:
print(global_music_file_paths[:4])

['spotify za v2\\regional-za-weekly-2018-03-15.csv', 'spotify za v2\\regional-za-weekly-2018-03-22.csv', 'spotify za v2\\regional-za-weekly-2018-03-29.csv', 'spotify za v2\\regional-za-weekly-2018-04-05.csv']


In [5]:
# global_music_file_paths

In [6]:
#Add the dates to the files - run once
def addsDatesToData():
    to_remove_front = "spotify za v2\\regional-za-weekly-"
    to_remove_back = ".csv"

    len_front_remove = len(to_remove_front)
    len_back_remove = len(to_remove_back)

    for file in global_music_file_paths: 
#         print(file)
        the_data_in_file = pd.read_csv(file)
#         the_data_in_file = the_data_in_file.iloc[1: , :]
#         the_data_in_file.columns = ["Position", "Track Name", "Artist", "Streams", "URL"]
        
        
        the_dates = file[len_front_remove:]
        the_dates = the_dates[:-len_back_remove]

        split_dates = the_dates.split("-")

        start_date = split_dates[0] + "-" + split_dates[1] + "-" + split_dates[2]
        

        len_dates = the_data_in_file.shape[0]
        start_dates = len_dates * [start_date]

        date_columns = {"End Date": start_dates}   

        the_data_in_file["End Date"] = start_dates

        #write the data back to the File
        the_data_in_file.to_csv(file, index=False)
        
#remove dates in existing files
def removeExistingDates():
    for file in global_music_file_paths:
        
        the_data_in_file = pd.read_csv(file)

        the_data_in_file.drop(["Start Date", "End Date"], axis=1, inplace=True) 

        the_data_in_file.to_csv(file, index=False)
    
#check for dates ... opting for removal and putting them back on


In [7]:
# removeExistingDates()
addsDatesToData()

In [8]:
#Combine the files and sort by dates
all_files = []
trouble_files = []
for file in global_music_file_paths:
    the_data_in_file = pd.read_csv(file)
    
    if("URL" in list(the_data_in_file.columns)):
    
        the_data_in_file = the_data_in_file.drop(["URL"], axis=1)
    
    list_vals = sum(list(the_data_in_file.isnull().sum()))
    
    if(list_vals > 0):
        trouble_files.append(file)
    all_files.append(the_data_in_file)
    
the_spotify_data = pd.concat(all_files) 
# trouble_files_df = pd.concat(trouble_files)

In [9]:
# the_spotify_data["Start Date"].unique().tolist()

In [10]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [11]:
the_spotify_data.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'weeks_on_chart', 'streams', 'End Date'],
      dtype='object')

In [12]:
the_spotify_data

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date
0,1,spotify:track:2XW4DbS6NddZxRPm5rMCeY,Drake,God's Plan,Cash Money/Drake LP6,1,-1,1,10313,2018-03-15
1,2,spotify:track:65NwOZqoXny4JxqAPlfxRF,"Post Malone, Ty Dolla $ign",Psycho (feat. Ty Dolla $ign),Republic Records,2,-1,1,6588,2018-03-15
2,3,spotify:track:2U7Hu2RaNQNRi7izoJyZaX,"Post Malone, 21 Savage",rockstar (feat. 21 Savage),Republic Records,3,-1,1,5738,2018-03-15
3,4,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,"Kendrick Lamar, SZA",All The Stars (with SZA),Black Panther (TDE/DMG) PS,4,-1,1,5585,2018-03-15
4,5,spotify:track:45Egmo7icyopuzJN0oMEdk,"Khalid, Normani",Love Lies (with Normani),RCA Records Label,5,-1,1,5535,2018-03-15
...,...,...,...,...,...,...,...,...,...,...
195,196,spotify:track:4C9q7Zd0RtMoZgh1ptIAbw,"Wanitwa Mos, Nkosazana Daughter, Master KG, Da...",Keneilwe (feat. Dalom Kids),Wanitwa Mos Entertainment,2,186,48,88679,2024-10-17
196,197,spotify:track:2qxgejJTaZIHNSHDD22Uhl,"Kabza De Small, DJ Maphorisa, Ami Faku",Abalele,Piano Hub,1,189,133,87892,2024-10-17
197,198,spotify:track:76Updb7OaDsoOHuqFbsuYr,Elaine,Risky,Columbia,45,-1,50,87795,2024-10-17
198,199,spotify:track:6OJblc63vbMcC9uEIY1Hpq,"Da Mabusa, Ama Grootman",Ithuba,Major Role Music,97,143,11,87706,2024-10-17


### Error checking that all the Thursday dates are present

In [13]:
date_list = the_spotify_data["End Date"].unique().tolist()
start_date = date_list[0]
end_date = date_list[len(date_list)-1]

In [14]:
len(date_list)

345

In [15]:
start_date, end_date

('2018-03-15', '2024-10-17')

In [16]:
def check_thursdays_in_range(date_list, start_date, end_date):
    # Convert start_date and end_date strings to datetime objects
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Initialize a set to store Thursdays in the range
    thursdays_in_range = set()
    
    # Iterate through the range of dates and add Thursdays to the set
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 3:  # Thursday
            thursdays_in_range.add(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    
    # Convert date_list to a set for efficient membership checking
    date_set = set(date_list)
    
    # Find missing Thursdays
    missing_thursdays = sorted(thursdays_in_range - date_set)
    print("thursdays_in_range: ", len(thursdays_in_range))
    print("date_set: ", len(date_set))
    
    if missing_thursdays:
        print("Missing Thursdays in the range:")
        for thursday in missing_thursdays:
            print(thursday)
        return False
    else:
        print("All Thursdays are present in the range.")
        return True

result = check_thursdays_in_range(date_list, start_date, end_date)
print("All Thursdays present:", result)


thursdays_in_range:  345
date_set:  345
All Thursdays are present in the range.
All Thursdays present: True


In [17]:
# the_spotify_data["End Date"].unique().tolist()

In [18]:
the_spotify_data["Week"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["Year"] = the_spotify_data["Week"]

the_spotify_data['Week'] = the_spotify_data['Week'].apply(lambda x : x.isocalendar()[1] )
the_spotify_data['Year'] = the_spotify_data['Year'].apply(lambda x : x.isocalendar()[0] )

# the_spotify_data["Start Date Dt"] = the_spotify_data["Start Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["End Date Dt"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data['Seconds since Epoch'] = the_spotify_data['End Date Dt'].apply(lambda x : round(x.timestamp(), 0))

In [19]:
#get the month when this happened
def changedaytotheFirst(x):
    x = x[:-2] + "01"
    
    return x

In [20]:
the_spotify_data["End Date Aggregated"] = the_spotify_data["End Date"].apply(lambda x : changedaytotheFirst(x) )

In [21]:
#find the number of artists on the track
the_spotify_data.dropna(inplace=True)

the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["artist_names"] = the_spotify_data.apply(lambda x : x["artist_names"] + "Artist_NA" if(x["artist_names"] == "") else x["artist_names"], axis=1)

the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["track_name"] = the_spotify_data.apply(lambda x : x["artist_names"] + "_NA" if(x["track_name"] == "") else x["track_name"], axis=1)

def countNumberArtists(x):
    len_all_featured_artists = 0
    num_artists = 0
    xvals = x.split("feat")
    len_xvals = len(xvals)
    
    if(len_xvals > 1):
        featured_artists = xvals[1:]
        featured_artists_str = ' '.join(featured_artists)
        all_featured_artists = featured_artists_str.split("& ")
        len_all_featured_artists = len(all_featured_artists)
        
    num_artists = 1 + len_all_featured_artists
    
    return num_artists
        
    
the_spotify_data['ArtistCount'] = the_spotify_data['track_name'].apply(lambda x : countNumberArtists(x))

In [22]:
the_spotify_data.shape

(68886, 16)

In [23]:
the_spotify_data["main_artist"] = the_spotify_data.apply(lambda x : x["artist_names"].split(",")[0], axis=1)

In [24]:
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["index"], axis=1, inplace=True)

In [25]:
the_spotify_data.shape

(68886, 17)

In [26]:
def everInTopTen(x):
    position_vals = list(x["rank"])
    position_vals = list(set(position_vals))
    
#     print(position_vals)
    
    min_pos = min(position_vals)
    existence_val = 0
    
    if(min_pos <= 10):
        existence_val = 1
    
    x["isTopTen"] = [existence_val] * len(x)
    
    return x
    
the_spotify_data = the_spotify_data[~the_spotify_data.index.duplicated()]
the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : everInTopTen(x))

In [27]:
the_spotify_data.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date,Week,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen
0,1,spotify:track:2XW4DbS6NddZxRPm5rMCeY,Drake,God's Plan,Cash Money/Drake LP6,1,-1,1,10313,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,1,Drake,1
1,2,spotify:track:65NwOZqoXny4JxqAPlfxRF,"Post Malone, Ty Dolla $ign",Psycho (feat. Ty Dolla $ign),Republic Records,2,-1,1,6588,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,2,Post Malone,1
2,3,spotify:track:2U7Hu2RaNQNRi7izoJyZaX,"Post Malone, 21 Savage",rockstar (feat. 21 Savage),Republic Records,3,-1,1,5738,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,2,Post Malone,1
3,4,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,"Kendrick Lamar, SZA",All The Stars (with SZA),Black Panther (TDE/DMG) PS,4,-1,1,5585,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,1,Kendrick Lamar,1
4,5,spotify:track:45Egmo7icyopuzJN0oMEdk,"Khalid, Normani",Love Lies (with Normani),RCA Records Label,5,-1,1,5535,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,1,Khalid,1


### When did the track reach the Top 10 

In [28]:
the_spotify_data.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date,Week,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen
0,1,spotify:track:2XW4DbS6NddZxRPm5rMCeY,Drake,God's Plan,Cash Money/Drake LP6,1,-1,1,10313,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,1,Drake,1
1,2,spotify:track:65NwOZqoXny4JxqAPlfxRF,"Post Malone, Ty Dolla $ign",Psycho (feat. Ty Dolla $ign),Republic Records,2,-1,1,6588,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,2,Post Malone,1
2,3,spotify:track:2U7Hu2RaNQNRi7izoJyZaX,"Post Malone, 21 Savage",rockstar (feat. 21 Savage),Republic Records,3,-1,1,5738,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,2,Post Malone,1
3,4,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,"Kendrick Lamar, SZA",All The Stars (with SZA),Black Panther (TDE/DMG) PS,4,-1,1,5585,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,1,Kendrick Lamar,1
4,5,spotify:track:45Egmo7icyopuzJN0oMEdk,"Khalid, Normani",Love Lies (with Normani),RCA Records Label,5,-1,1,5535,2018-03-15,11,2018,2018-03-15,1.521072e+09,2018-03-01,1,Khalid,1


In [29]:
the_spotify_data["isTopTen"].value_counts()

0    46591
1    22295
Name: isTopTen, dtype: int64

In [30]:
the_spotify_data.isnull().sum()

rank                   0
uri                    0
artist_names           0
track_name             0
source                 0
peak_rank              0
previous_rank          0
weeks_on_chart         0
streams                0
End Date               0
Week                   0
Year                   0
End Date Dt            0
Seconds since Epoch    0
End Date Aggregated    0
ArtistCount            0
main_artist            0
isTopTen               0
dtype: int64

In [31]:
unique_artists = the_spotify_data["main_artist"].unique().tolist()
num_artists = len(unique_artists)
print(num_artists)

1202


In [32]:
### find new artists in that week

In [33]:
# the_spotify_data.head(12)
def getArtistAppearanceCount(x):
    
    x = x.reset_index()
    x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'artistAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x        
    
the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)

In [34]:
the_spotify_data.head()

,artistAppearanceCount,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date,Week,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen
0,0,14,spotify:track:3w0w2T288dec0mgeZZqoNN,"$, Kanye West, Ty Dolla $ign",CARNIVAL,Ye,14,-1,1,203139,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0
1,1,30,spotify:track:347AQK5Lyhn6RvB8tBGYxt,"$, Kanye West, Ty Dolla $ign",STARS,Ye,30,-1,1,143990,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0
2,2,31,spotify:track:1eaqMiiUn2P7MnqJK4XeK0,"$, Kanye West, Ty Dolla $ign",TALKING,Ye,31,-1,1,143905,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0
3,3,38,spotify:track:1icgLGTpX2fQXKRe4D7w2b,"$, Kanye West, Ty Dolla $ign",BACK TO ME,Ye,38,-1,1,133735,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0
4,4,49,spotify:track:5tEaVciE2GnR28aN6W9cLS,"$, Kanye West, Ty Dolla $ign",FUK SUMN,Ye,49,-1,1,114081,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0


In [35]:
def getTrackAppearanceCount(x):
    
    x = x.reset_index()
    x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'trackAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x         
    
the_spotify_data =the_spotify_data.groupby(["artist_names", "track_name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)

In [36]:
the_spotify_data.head()

,trackAppearanceCount,artistAppearanceCount,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date,Week,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen
0,0,3,38,spotify:track:1icgLGTpX2fQXKRe4D7w2b,"$, Kanye West, Ty Dolla $ign",BACK TO ME,Ye,38,-1,1,133735,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0
1,1,15,85,spotify:track:1icgLGTpX2fQXKRe4D7w2b,"$, Kanye West, Ty Dolla $ign",BACK TO ME,Ye,38,38,2,88457,2024-02-22,8,2024,2024-02-22,1.708560e+09,2024-02-01,1,$,0
2,0,7,74,spotify:track:4ihV1wv7QmjbkLHaT9lA4s,"$, Kanye West, Ty Dolla $ign",BEG FORGIVENESS,Ye,74,-1,1,95997,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0
3,0,8,79,spotify:track:04CyMEHliadfQWMUJb1w99,"$, Kanye West, Ty Dolla $ign",BURN,Ye,79,-1,1,91850,2024-02-15,7,2024,2024-02-15,1.707955e+09,2024-02-01,1,$,0
4,1,17,117,spotify:track:04CyMEHliadfQWMUJb1w99,"$, Kanye West, Ty Dolla $ign",BURN,Ye,79,79,2,79620,2024-02-22,8,2024,2024-02-22,1.708560e+09,2024-02-01,1,$,0


In [37]:
the_spotify_data.to_csv("the_spotify_data.csv")

In [38]:
the_spotify_data["rank"] = the_spotify_data["rank"].astype(int)
the_spotify_data["streams"] = the_spotify_data["streams"].astype(int)
the_spotify_data["ArtistCount"] = the_spotify_data["ArtistCount"].astype(int)

the_spotify_data["isTopTen"] = the_spotify_data["isTopTen"].astype(int)
# the_spotify_data["lenOnCharts"] = the_spotify_data["lenOnCharts"].astype(int)
the_spotify_data["Year"] = the_spotify_data["Year"].astype(int)
the_spotify_data["Week"] = the_spotify_data["Week"].astype(int)

In [39]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [40]:
the_spotify_data["Artist and Track"] = the_spotify_data["artist_names"] + "; " + the_spotify_data["track_name"]

### Getting the list of Artists and Tracks

In [41]:
import pandas as pd
# import pymongo
# import getlyrics

In [42]:
def trackAppearance(x):
#     x = x.reset_index()

    x = x.sort_values(by=["End Date"])
#     x.set_index(["Position"])

    x["rank"] = x["rank"].astype(int)
#     print(list(x["Position"]))
    
    x["rank difference"] = x["rank"].diff().fillna(0)
    x["rank difference"] = x["rank difference"].astype(int)
    
#     x["Position Difference"] = x["Position"].sub(x["Position"].shift())
#     x.drop(["index"], inplace=True)
    
    return x
    
all_files_grouped = the_spotify_data.groupby(["main_artist", "track_name"]).apply(lambda x: trackAppearance(x))

In [43]:
all_files_grouped["rank difference"].max()

187

### Position over time

In [44]:
def positionvertime(x):
    posovertime = 0
    #new track
    if(x["rank difference"] == 0 and x["trackAppearanceCount"]==0):
        posovertime = 0
    #track stayed in the same position
    elif(x["rank difference"] == 0 and x["trackAppearanceCount"]!=0):
        posovertime = 50
    #track went up the chart
    elif(x["rank difference"] < 0):
        posovertime = 75
    #track fell down the chart
    else:
        posovertime = 100
    return posovertime

In [45]:
all_files_grouped["Position over Time"] = all_files_grouped.apply(lambda x : positionvertime(x), axis=1)

##### Add a Girl Group and Boy Group

In [46]:
ggdf = pd.read_csv("Classification/ListofGirlGroups.csv", encoding='ANSI')
bgdf = pd.read_csv("Classification/ListofBoyGroups.csv", on_bad_lines='skip', encoding='ANSI')

In [47]:
def IsGG(ggdf, y):
    ggdf["Artist"] = ggdf["Artist"].astype(str)
    ggs = [x.lower().strip() for x in ggdf["Artist"]]
    y = y.lower().strip()
    
    if y in ggs:
        return True
    else:
        return False

def IsBG(bgdf, y):
    bgdf["Artist"] = bgdf["Artist"].astype(str)
    bgs = [x.lower().strip() for x in bgdf["Artist"]]
    y = y.lower().strip()
    
    if y in bgs:
        return True
    else:
        return False

In [48]:
ggdf

,Artist,Debut Year,Members
0,257,NaN,NaN
1,(G)I-dle,NaN,NaN
2,+(KR)ystal Eyes,NaN,NaN
3,2Eyes,NaN,NaN
4,2NE1,NaN,NaN
...,...,...,...
363,NaN,2024.0,NaN
364,NaN,2024.0,NaN
365,NaN,2024.0,NaN
366,NaN,2024.0,NaN


In [49]:
all_files_grouped["IsGirlGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(ggdf, x))
all_files_grouped["IsBoyGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(bgdf, x))

ggs = all_files_grouped[all_files_grouped["IsGirlGroup"]==True]["main_artist"].unique().tolist()
bgs = all_files_grouped[all_files_grouped["IsBoyGroup"]==True]["main_artist"].unique().tolist()

In [50]:
all_files_grouped.to_csv("Classification/all_files_ZAv1.csv", index=False)

In [51]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [52]:
# new_all_files_grouped = all_files_grouped.reset_index(drop=True)

In [53]:
def newstuff(x):
#     x = x.reset_index()

    x = x.sort_values(by=['Seconds since Epoch'])
#     x.set_index(["Position"])
    print(x)
    
    return x
    


### South African Artists

In [54]:
all_files_grouped.shape

(68886, 25)

In [55]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [56]:
narrowing_df = all_files_grouped[['End Date', "rank difference"]]

In [57]:
narrowing_df.reset_index(inplace=True)

In [58]:
narrowing_df.drop(["level_2"], axis=1, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_21088\3213521647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrowing_df.drop(["level_2"], axis=1, inplace=True)


In [59]:
narrowing_df = narrowing_df[narrowing_df["rank difference"] == 0]

In [60]:
narrowing_df.to_csv("narrow_uniq.csv", index=False)

In [61]:
# narow_artists_unique

In [62]:
uniquartists_cat_df = pd.read_csv("Classification/uniquartists_cat.csv")

In [63]:
narrowing_df_art = narrowing_df[["main_artist"]]
uniquartists_cat_df_art = uniquartists_cat_df[["main_artist"]]

narrowing_df_art.drop_duplicates(inplace=True)
uniquartists_cat_df_art.drop_duplicates(inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_21088\1985272047.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrowing_df_art.drop_duplicates(inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_21088\1985272047.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniquartists_cat_df_art.drop_duplicates(inplace=True)


In [64]:
len(narrowing_df_art), len(uniquartists_cat_df_art)

(1202, 1264)

In [65]:
minus_list = list(set(narrowing_df_art["main_artist"]) - set(uniquartists_cat_df_art["main_artist"]))

In [66]:
minus_list_df = pd.DataFrame()
minus_list_df["main_artist"] = minus_list
minus_list_df["country"] = [""]*len(minus_list)
minus_list_df.to_csv("Classification/New Artist to Cat.csv", index=False)

In [67]:
edited_minus_list_df = pd.read_csv("Classification/New Artist to Cat.csv")
uniquartists_cat_df = pd.read_csv("Classification/uniquartists_cat.csv")

added_files = [edited_minus_list_df, uniquartists_cat_df]
added_files_df = pd.concat(added_files)
added_files_df.to_csv("Classification/uniquartists_cat.csv", index=False)

In [68]:
south_african_artists_df = added_files_df[added_files_df["country"] == "ZA"]
south_african_artists = south_african_artists_df["main_artist"].tolist()

In [69]:
len(south_african_artists)

414

In [70]:
south_african_artists_lower = []
for sartists in south_african_artists:
    south_african_artists_lower.append(sartists.lower())

In [71]:
uniq_sans = south_african_artists_lower

In [72]:
len(uniq_sans)

414

In [73]:
len(south_african_artists)

414

In [74]:
all_files_grouped['Lower Artist'] = all_files_grouped['main_artist'].apply(lambda x : x.lower() )

In [75]:
boolean_series = all_files_grouped["Lower Artist"].isin(uniq_sans)
all_files_grouped_filtered_df = all_files_grouped[boolean_series]

In [76]:
all_files_grouped_filtered_df.to_csv("Classification/JustSouthAfricansOnZAChart.csv", index=False)

In [77]:
all_files_grouped_filtered_df.drop(["main_artist", "track_name"], inplace=True, axis=1)

C:\Users\User\AppData\Local\Temp\ipykernel_21088\2304093722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_files_grouped_filtered_df.drop(["main_artist", "track_name"], inplace=True, axis=1)


In [78]:
all_files_grouped_filtered_df = all_files_grouped_filtered_df.reset_index()

In [79]:
all_files_grouped_filtered_df.drop(["level_2"], axis=1, inplace = True)

In [80]:
tot_streams_all_files_grouped_filtered_df = all_files_grouped_filtered_df[['main_artist', 'streams']]

In [81]:
tot_streams_all_files_grouped_filtered_df_fin = tot_streams_all_files_grouped_filtered_df.groupby(["main_artist"]).sum()

In [82]:
tot_streams_all_files_grouped_filtered_df_fin.reset_index(inplace=True)

In [83]:
tot_streams_all_files_grouped_filtered_df_fin = tot_streams_all_files_grouped_filtered_df_fin.sort_values(by='streams', ascending=False)

In [84]:
tot_streams_all_files_grouped_filtered_df_fin.head(11)

,main_artist,streams
151,Kabza De Small,94631699
157,Kelvin Momo,51010299
367,Tyler ICU,38306338
225,Mellow & Sleazy,34583355
63,DJ Stokie,34444440
195,Lwah Ndlunkulu,33684074
381,Wanitwa Mos,31055855
303,Sam Deep,29096093
93,Dlala Thukzin,26116718
214,Mas Musiq,24527460


In [85]:
top_10_ZA_artists = tot_streams_all_files_grouped_filtered_df_fin["main_artist"].tolist()

In [86]:
# top_10_ZA_artists

In [87]:
for artist in top_10_ZA_artists:
    kabza_de_small = all_files_grouped_filtered_df[all_files_grouped_filtered_df["main_artist"] == artist]

    kb_racing_bar_df = kabza_de_small[['End Date Aggregated', "track_name", 'Year', 'streams']]

    kb_racing_bar_df.columns = ['date', "name", 'category', 'value']
    kb_racing_bar_df = kb_racing_bar_df.groupby(['date', "name", 'category']).sum()
    kb_racing_bar_df.reset_index(inplace=True)
    
    kb_racing_bar_df.to_csv("Classification/topten/_" + artist + "_ZAChartracing_bar.csv", index=False)

### For the bar chart race

In [88]:
len(all_files_grouped_filtered_df)

16685

In [89]:
racing_bar_df = all_files_grouped_filtered_df[['End Date Aggregated', "main_artist", 'Year', 'streams']]

In [90]:
racing_bar_df.columns = ['date', "name", 'category', 'value']

In [91]:
racing_bar_df.to_csv("Classification/JustSouthAfricansOnZAChartracing_bar.csv", index=False)

In [92]:
# len(racing_bar_df["End Date Dt"].unique().tolist())

#### Monthly bar chart race

In [93]:
monthly_racing_bar_df = all_files_grouped_filtered_df[['End Date Dt', "main_artist", 'Year', 'streams']]

# monthly_racing_bar_df.groupby(["Artist", 'End Date Dt']).sum('Streams')

In [94]:
monthly_racing_bar_df["Month"] = monthly_racing_bar_df["End Date Dt"].apply(lambda x : x.month)

C:\Users\User\AppData\Local\Temp\ipykernel_21088\1086615051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_racing_bar_df["Month"] = monthly_racing_bar_df["End Date Dt"].apply(lambda x : x.month)


In [95]:
monthly_racing_bar_agg_df = monthly_racing_bar_df.groupby(["main_artist", "Month", "Year"]).sum("streams").reset_index()

In [96]:
monthly_racing_bar_agg_df["date"] = monthly_racing_bar_agg_df.apply( lambda x : datetime(int(x["Year"]), int(x["Month"]), 1), axis=1)

In [97]:
monthly_racing_bar_agg_df.drop(["Year"], inplace = True, axis=1)

In [98]:
monthly_racing_bar_fin = pd.DataFrame()
monthly_racing_bar_fin['date'] = monthly_racing_bar_agg_df["date"]
monthly_racing_bar_fin['name'] = monthly_racing_bar_agg_df["main_artist"]
monthly_racing_bar_fin['category'] = monthly_racing_bar_agg_df["Month"]
monthly_racing_bar_fin['value'] = monthly_racing_bar_agg_df["streams"]

In [99]:
monthly_racing_bar_fin.to_csv("Classification/JustSouthAfricansOnZAChartMonthlyracing_bar.csv", index=False)